In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from pykat import finesse
from pykat.commands import *
from IPython.display import display, HTML
%matplotlib inline
pykat.init_pykat_plotting(dpi=90)
basekat=finesse.kat() #Initialize Finesse
basekat.verbose = False
basecode = """
l laser 1 0 n0 # Laser (P=1W,lambda offset = 0)
s s1 1 n0 nc1 # Space (length = 1m)
##Cavity
m m1 0.7 0.3 0 nc1 nc2 # Mirror (R = 0.7, T=0.3, phi=0)
s sL 4000 nc2 nc3 #space length 4km
m m2 0.8 0.2 0 nc3 nc4 #Mirror (R=0.8,phi=0)
"""
basekat.parseCommands(basecode)
#Simulation instructions
kat1=basekat.deepcopy()
code='''
##Detectors
#Photodiodes measuring DC
pd refl nc1
pd circ nc2
pd tran nc4

#Instructions
xaxis laser f lin -5k 90k 1000
yaxis abs
'''
kat1.parseCommands(code)
#1st run, original cavity length
out1a = kat1.run()
#2nd run, length 6km
kat1.sL.L=6000
out1b=kat1.run()
#3rd Run, length 8km
kat1.sL.L=8000
out1c=kat1.run()
# Run 1
fig1a = out1a.plot(xlabel='Frequency [Hz]', 
                   ylabel='Power [W]', 
                   title='Run 1: Cavity length = 4 km',
                   loc=1)
# Run 2
fig1b = out1b.plot(xlabel='Frequency [Hz]', 
                   ylabel='Power [W]', 
                   title='Run 2: Cavity length = 6 km', 
                   loc=1)
# Run 3
fig1c = out1c.plot(xlabel='Frequency [Hz]', 
                   ylabel='Power [W]', 
                   title='Run 3: Cavity length = 8 km', 
                   loc = 1)

                                              ..-
    PyKat 1.0.17          _                  '(
                          \`.|\.__...-""""-_." )
       ..+-----.._        /  ' `            .-'
   . '            `:      7/* _/._\    \   (
  (        '::;;+;;:      `-"' =" /,`"" `) /
  L.        \`:::a:f            c_/     n_'
  ..`--...___`.  .    ,  
   `^-....____:   +.      www.gwoptics.org/pykat



We wish to investigate how free spectral range (dist between peaks) depends on cavity length by testing different lengths. We do this using scipy.optimize.
1. Copy basekat add a photodiode to measure transmitted power (use noaxis to get only one data point for the function to work)


In [2]:
kat2 = basekat.deepcopy()
code = """
##Detectors
pd tran nc4
##Simulate
noxaxis
yaxis abs
"""
kat2.parseCommands(code)
import scipy.optimise as op
def find_fsr(kat):
    #Define a function for finding FSR
    laser = kat.getAll(pykat.components.laser)[0]
    def find_max(f,df):
        #Minimise at f +df
        laser.f = f + df
        out_max = kat.run()
        return -out_max.y[0,0]
    def find_min(f,df):
        laser.f= f + df
        out_min = kat.run()
        return out_min.y[0,0]
    #Find first peak
    offset = 0 #guess df = 0
    xtol = 1.0e-6
    opts = {'xtol': xtol, 'maxiter': 500}
    #Minimize scalar with brent's method
    sol_1 = op.minimize_scalar(find_max, args=(offset), method='brent', options=opts)
    max1 = sol_1.x #Store position

    #Estimate minimum between peaks

    xtol = 1.0e-1 #Rough estimation using higher tolerance
    opts = {'xtol': xtol, 'maxiter':500}
    offset = max1+10*xtol #start a bit away from max1
    #Call minimize_scalar to find minimum of find_min()
    sol_2 = op.minimize_scalar(find_min,args=(offset),method='brent',options=opts)
    min1 = sol_2.x + offset
    #Find second peak
    xtol = 1.0e-6 #lower tolerance
    opts = {'xtol': xtol, 'maxiter':500}
    offset = 2*min1-max1 #start guess based on previous
    sol_3 = op.minimize_scalar(find_max,args=(offset),method='brent',options=opts)
    max2 = sol_3.x + offset #store peak position

    fsr = np.abs(max2-max1) #Compute FSR
    print('L = {0:.1f}m, FSR = {1:.2f} Hz'.format(kat.sL.L.value, fsr))

    return fsr

ImportError: No module named 'scipy.optimise'